# I. Import packages

In [2]:
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

# II Input Data and create utility functions

In [3]:
data_address = "data.xlsx"

def lst_to_dict(lst):
    mydict = dict()
    for sublist in lst:
        mydict[tuple(sublist[0:3])] = sublist[len(sublist) - 1]
    return mydict


def _2d_lst_to_dict(lst, V):
    mydict = dict()
    for v in V:
        for i in range(len(lst)):
            for j in range(len(lst)):
                mydict[(i, j, v)] = lst[i][j]
    return mydict

def p_arr_to_m_dict(arr, n, r):
    m_dict = {}
    for key1, key2, value, _ in arr:
        if (key1 <= n) and (key2 <= r):
            if (key1, key2) in m_dict:
                m_dict[(key1, key2)].append(value)
            else:
                m_dict[(key1, key2)] = [value]

    return m_dict


# III. Define Model

In [4]:
def MIP_solver(data_address):
    df = pd.read_excel(data_address, "set")

    n = int(df.iloc[0, 2])
    r = int(df.iloc[1, 2])
    m = int(df.iloc[2, 2])
    v = int(df.iloc[3, 2])
    # set # index [1, n+1]
    N_full = range(0,n+2)
    N = range(1, n+1)
    N_plus = range(1,n+2)
    N_minus = range(0, n+1)
    R = range(1, r+1)
    R_minus = range(2, r+1)
    M = range(1, m+1)
    V = range(1, v+1)


    # Parameter
    # unit processing cost on machine m per unit of time
    lamda = np.transpose(np.delete(np.array(pd.read_excel(
        data_address, "machine_cost")), 0, 1))[0]

    # fixed cost of vehicle type v
    F = np.transpose(np.delete(np.array(pd.read_excel(
        data_address, "vehicle_cost")), 0, 1))[0]

    # variable cost of vehicle v per unit of time
    Theta = np.transpose(np.delete(np.array(pd.read_excel(
        data_address, "vehicle_cost")), 0, 1))[1]

    # weight of early delivery
    mu = df.iloc[4, 2]

    # weight of tardy delivery
    fei = df.iloc[5, 2]

    # process time of operation r of job j on machine m
    # [job, operation, machine, time]
    P = np.array(pd.read_excel(
        data_address, "processing_time"))

    m_dict = p_arr_to_m_dict(P, n, r)
    P = lst_to_dict(P)

    print("machine_dict", m_dict)
    print("Pro time", P)

    # process machine matrix
    # [job, operation, machine, value]
    a = lst_to_dict(np.array(pd.read_excel(
        data_address, "process_machine_matrix")))

    print("Pro value", a)


    # transportation time from customer i to customer j by vehicle v
    t = _2d_lst_to_dict(
        np.delete(np.array(pd.read_excel(
            data_address, "transport_time")), 0, 1), V)

    print("Tran time", t)
    # size of job j
    theta = np.transpose(np.delete(np.array(pd.read_excel(
        data_address, "size_job")), 0, 1))[0]
    print("job size", theta)

    # delivery time window of job j
    tw = np.array(pd.read_excel(
        data_address, "delivery_time_window"))[:,1:]
    print("time_window", tw[0,:])

    # Objective bound
    obj1_bounds = np.array(pd.read_excel(
        data_address, "pareto_front"))[0, 1:] * n/30 * np.random.randint(40, 55)/50

    obj2_bounds = np.array(pd.read_excel(
        data_address, "pareto_front"))[1, 1:] * n/30 * np.random.randint(40, 55)/50

    # capacity of vehicle v
    q = np.transpose(
        np.delete(np.array(pd.read_excel(
            data_address, "vehicle_capacity")), 0, 1))[0]
    print("capacity", q)

    # Very large number
    bigM = 1e15

    # decision variable
    # production

    solver = pywraplp.Solver.CreateSolver("SCIP")

    # production start time of operation r of job j
    pi = {}
    for j in N: 
        for r in R:
            pi[(j, r)] = solver.NumVar(0, solver.infinity(), "")

    # production completion time of operation r of job j
    gamma = {}
    for j in N_full:
        for r in R:
            gamma[(j, r)] = solver.NumVar(0, solver.infinity(), "")

    # production complation time of job j
    C = []
    for j in N:
        C.append(solver.NumVar(0, solver.infinity(), ""))

    # binary variable takes the value 1 if operation r of job j is processed by machine m else 0
    X = {}
    for j in N:
        for r in R:
            for m in m_dict[(j, r)]:
                X[(j, r, m)] = solver.BoolVar("")

    # binary variable takes the value 1 if operation r of job j is processed immediately after the operation f of job i, both on machine m. else 0
    Y = {}
    for i in N_full:
        for f in R:
            for j in N_full:
                for r in R:
                    for m in M:
                        if i != j:
                            Y[(i, f, j, r, m)] = solver.BoolVar("")
                            # Y[(i, f, j, r, m)] = solver.NumVar(0, 1, "")

    # distribution
    # deliver time of order (job) j
    D = []
    for j in N:
        D.append(solver.NumVar(0, solver.infinity(), ""))

    # visiting time of customer (order) j by vehicle v
    T = {}
    for j in N_full:
        for v in V:
            T[(j, v)] = solver.NumVar(0, solver.infinity(), "")

    # leaving time of vehicle v from production facility
    S = []
    for v in V:
        S.append(solver.NumVar(0, solver.infinity(), ""))

    # visiting time of the last customer (order) in the tour of vehicle v
    E = []
    for v in V:
        E.append(solver.NumVar(0, solver.infinity(), ""))

    # binary variable takes the value 1 if job j is delivered by vehicle else 0
    Z = {}
    for j in N:
        for v in V:
            # Z[(j, v)] = solver.BoolVar("")
            Z[(j, v)] = solver.NumVar(0, 1, "")

    # binary variable takes the value 1 if job j is delivered after job i, by vehicle v else 0
    U = {}
    for i in N_full:
        for j in N_full:
            for v in V:
                if i != j:
                    # U[(i, j, v)] = solver.BoolVar("")
                    U[(i, j, v)] = solver.NumVar(0, 1, "")

    # binary variable take the value 1 if vehicle v is used for delivery else 0
    W = []
    for v in V:
        # W.append(solver.BoolVar(""))
        W.append(solver.NumVar(0, 1, ""))


    ## Constraint

    # 3
    for j in N:
        for r in R:
                solver.Add(solver.Sum([X[(j, r, m)] for m in m_dict[(j, r)]]) == 1, "ct3")

    # 4
    for j in N:
        for r in R:
            for m in m_dict[(j, r)]:
                solver.Add(X[(j, r, m)] <= a[(j, r, m)], "ct4")


    # 5
    for i in N:
        for f in R:
            for m in m_dict[(i, f)]:
                solver.Add(X[(i, f, m)] == solver.Sum(
                    [Y[(i,f,j,r,m)] for j in N_plus for r in R if i != j]), "ct5")


    # 6
    for j in N:
        for r in R:
            for m in m_dict[(j, r)]:
                solver.Add(X[(j, r, m)] == solver.Sum(
            [Y[(i,f,j,r,m)] for i in N_minus for f in R if i != j]), "ct6")

    # 7 Linear max
    bc7 = {}
    for i in N:
        for f in R:
            for m in m_dict[(i, f)]:
                bc7[(i, f, m)] = solver.NumVar(0, bigM, "")

    z7 = {}
    for j in N:
        for r in R_minus:
            for i in range(2):
                z7[(j, r, i)] = solver.BoolVar("")
                
    for j in N:
        for r in R_minus:
            solver.Add(pi[(j, r)] >= gamma[(j,r-1)])
            solver.Add(pi[(j, r) ] <= gamma[(j,r-1)] + bigM*z7[(j, r, 0)])

            for i in N:
                for f in R:
                    for m in m_dict[(i, f)]:
                        if i != j:
                            solver.Add(bc7[(i, f, m)] <= bigM * Y[(i, f, j, r, m)])
                            solver.Add(bc7[(i, f, m)] <= gamma[(i,f)])
                            solver.Add(bc7[(i, f, m)] >= gamma[(i,f)] - bigM*(1-Y[(i, f, j, r, m)]))
            solver.Add(pi[(j, r)] >= solver.Sum([bc7[(i, f, m)] for i in N for f in R for m in m_dict[(i, f)] if i != j]))

    # 8
    for j in N:
        for r in R:
            solver.Add(gamma[(j, r)] == pi[(j,r)] + solver.Sum([P[(j, r, m)] * X[(j,r,m)] for m in m_dict[(j, r)]]), "ct8")

    # 9
    for j in N:
        solver.Add(C[j-1] == gamma[(j, R[-1])], "ct9")


    # 10
    for m in M:
        solver.Add(solver.Sum(
            [Y[(0, f, j, r, m)] for j in N for f in R for r in R]) == 1, "ct10")

    # 11
    for m in M:
        solver.Add(solver.Sum(
            [Y[(i, f, n+1, r, m)] for i in N for f in R for r in R]) == 1, "ct11")
        
        


    # 12
    for j in N:
        solver.Add(solver.Sum(
            [Z[(j, v)] for v in V]) == 1, "ct12")
        
    # 13
    for j in N:
        for v in V:
            solver.Add(Z[(j,v)] == solver.Sum(
                [U[((i, j, v))] for i in N_minus if i !=j]), "ct13")

    # 14 
    for j in N:
        for v in V:
            solver.Add(solver.Sum(
                [U[(i,j,v)] for i in N_minus if i != j]) == solver.Sum(
                    [U[(i,j,v)] for i in N_plus if i != j]), "ct14a")
            solver.Add(solver.Sum(
                    [U[(i,j,v)] for i in N_plus if i != j]) <= 1, "ct14b")
            
    # 15 
    for v in V:
        solver.Add(solver.Sum(
            [U[(0,j,v)] for j in N]) == solver.Sum(
                [U[(i,n+1,v)] for i in N]), "ct15a")
        solver.Add(solver.Sum(
                [U[(i,n+1,v)] for i in N]) <= 1, "ct15b")
        
    # 16
    for v in V:
        solver.Add(solver.Sum(
            [Z[(j, v)]*theta[j-1] for j in N]) <= q[v-1], "ct16")
        
    # 17 Add max ct
    # Linearize binary * continuous
    z17 = {}
    for v in V:
        for j in N:
                z17[(v, j)] = solver.BoolVar("")
    bc17 = {}
    for j in N:
        for v in V:
            bc17[(j,v)] = solver.NumVar(0, bigM, "")

    for v in V:
        solver.Add(T[(0, v)] == S[v-1], "ct15a")
        
        for j in N:
            solver.Add(bc17[(j,v)] <= bigM * Z[(j, v)])
            solver.Add(bc17[(j,v)] <= C[j-1])
            solver.Add(bc17[(j,v)] >= C[j-1] - bigM *(1 - Z[j, v]))

            solver.Add(S[v-1] >= bc17[(j,v)])
            solver.Add(S[v-1] <= bc17[(j,v)] + bigM*z17[(v, j)])

        solver.Add(solver.Sum([z17[(v, j)] for j in N]) <= n - 1)
        
    # 18 Linearize binary * continuous
    bc18 = {}

    for i in N:
        for j in N:
            for v in V:
                if i != j:
                    bc18[(i,j,v)] = solver.NumVar(0, bigM, "")

    for j in N:
        for v in V:
            for i in N:
                if i != j:
                    solver.Add(bc18[(i,j,v)] <= bigM * U[(i,j,v)])
                    solver.Add(bc18[(i,j,v)] <= T[(i,v)] + t[(i,j,v)])
                    solver.Add(bc18[(i,j,v)] >= T[(i,v)] + t[(i,j,v)] - bigM*(1-U[(i,j,v)]))
            solver.Add(T[(j,v)] == solver.Sum(
                    [bc18[(i,j,v)] for i in N if i != j]), "ct18")
            

    # 19 Linearize binary * continuous
    bc19 = {} # dummy variable
    for j in N:
        for v in V:
            bc19[(j,v)] = solver.NumVar(0, bigM, "")

    for j in N:
        for v in V:
            solver.Add(bc19[(j,v)] <= bigM * Z[j,v])
            solver.Add(bc19[(j,v)] <= T[(j,v)])
            solver.Add(bc19[(j,v)] >= T[(j,v)] - bigM*(1-Z[j,v]))

        solver.Add(D[j-1] == solver.Sum(
            [bc19[(j,v)] for v in V]), "ct19")

    # 20
    for v in V:
        solver.Add(E[v-1] == S[v-1] + solver.Sum(
            [U[(i,j,v)] * t[(i,j,v)] for i in N_minus for j in N_plus if i != j])
            , "ct20")

    # 21 Add Max constraint
    z21 = {}
    for v in V:
        for j in N:
                z21[(v, j)] = solver.BoolVar("")

    for v in V:        
        for j in N:
            solver.Add(W[v-1] >= Z[(j, v)])
            solver.Add(W[v-1] <= Z[(j, v)] + bigM*z21[(v, j)])

        solver.Add(solver.Sum([z21[(v, j)] for j in N]) <= n - 1)

    # 22 Max constraint for obj2

    ## Finish time
    max_fin = [0] * len(N)
    for j in N:
        max_fin[j-1] = solver.NumVar(0, bigM, "")

    for j in N:
        solver.Add(max_fin[j-1] >= D[j-1] - tw[j-1, 1], "")


    ## Beginning time
    max_beg = [0] * len(N)
    for j in N:
        max_beg[j-1] = solver.NumVar(0, bigM, "")

    for j in N:
        solver.Add(max_beg[j-1] >= tw[j-1, 0] - D[j-1], "")

    
    
    ProCost = solver.NumVar(0, bigM, "pro_cost")
    obj1_ProCost = solver.Add(ProCost == solver.Sum(lamda[m-1] * P[(j, r, m)] * X[(j, r, m)] for j in N for r in R for m in m_dict[(j, r)]))
    obj1_ProCost.set_is_lazy(laziness=True)

    DistCost = solver.NumVar(0, bigM, "DistCost")
    obj1_DistCost = solver.Add(DistCost == solver.Sum([F[v-1]*W[v-1] + Theta[v-1]*(E[v-1] - S[v-1]) for v in V]))
    obj1_DistCost.set_is_lazy(laziness=True)

    obj1 = solver.NumVar(0, bigM, "obj1")
    ct_obj1 = solver.Add(obj1 == ProCost\
                + DistCost, "ct_obj1")
    ct_obj1.set_is_lazy(laziness=True)


    obj2 = solver.NumVar(0, bigM, "obj2")
    ct_obj2 = solver.Add(obj2 == fei * solver.Sum([max_fin[j-1] for j in N])\
                        + mu * solver.Sum([max_beg[j-1] for j in N]), "ct_obj2")
    ct_obj2.set_is_lazy(laziness=True)


    # Sets a time limit of 1 hour.
    solver.SetTimeLimit(15*60*1000)

    # set a minimum gap limit for the integer solution during branch and cut
    gap = 0.25
    solverParams = pywraplp.MPSolverParameters()
    solverParams.SetDoubleParam(solverParams.RELATIVE_MIP_GAP, gap)

    # Initial solve
    print(f"Solving with {solver.SolverVersion()}")
    solver.Minimize(obj1)
    status = solver.Solve()

    # Print solution.

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f"Total cost = {obj1.solution_value()}")
        print(f"Total penalty = {obj2.solution_value()}\n")
            # binary variable takes the value 1 if operation r of job j is processed by machine m else 0
        for j in N:
            for r in R:
                for m in m_dict[(j, r)]:
                    if X[(j, r, m)].solution_value() == 1:
                        print(f"X[{j, r, m}] = 1") 

        # binary variable takes the value 1 if operation r of job j is processed immediately after the operation f of job i, both on machine m. else 0
        for i in N_full:
            for f in R:
                for j in N_full:
                    for r in R:
                        for m in M:
                            if i != j:
                                if Y[(i, f, j, r, m)].solution_value() == 1:
                                    print(f"Y[{i, f, j, r, m}] = 1") 

        # distribution

        # leaving time of vehicle v from production facility
        for v in V:
            print(f"S[{v}]", S[v-1].solution_value())

        # visiting time of the last customer (order) in the tour of vehicle v
        for v in V:
            print(f"E[{v}]", E[v-1].solution_value())

        # binary variable takes the value 1 if job j is delivered by vehicle else 0 
        for j in N:
            for v in V:
                if round(Z[(j, v)].solution_value()) == 1:
                    print(f"Z[{j, v}] = 1") 

        # binary variable takes the value 1 if job j is delivered after job i, by vehicle v else 0
        for i in N_full:
            for j in N_full:
                for v in V:
                    if i != j:
                        if round(U[(i, j, v)].solution_value()) == 1:
                            print(f"U[{i, j, v}] = 1") 

        # binary variable take the value 1 if vehicle v is used for delivery else 0
        for v in V:
            # W.append(solver.BoolVar(""))
            if round(W[v-1].solution_value()) == 1:
                print(f"W[{v}] = 1") 
        
    else:
        print("No solution found.")
        
    return solver, obj1, obj2, ct_obj1, ct_obj2, obj1_bounds, obj2_bounds, ProCost, DistCost
solver, obj1, obj2, ct_obj1, ct_obj2, obj1_bounds, obj2_bounds, ProCost, DistCost = MIP_solver(data_address)

machine_dict {(1, 1): [7], (1, 2): [7], (1, 3): [7], (1, 4): [7], (1, 5): [7], (1, 6): [7], (1, 7): [7], (2, 1): [8], (2, 2): [8], (2, 3): [8], (2, 4): [8], (2, 5): [8], (2, 6): [8], (2, 7): [8], (3, 1): [9], (3, 2): [9], (3, 3): [9], (3, 4): [9], (3, 5): [9], (3, 6): [9], (3, 7): [9], (4, 1): [1], (4, 2): [1], (4, 3): [1], (4, 4): [1], (4, 5): [1], (4, 6): [1], (4, 7): [1], (5, 1): [2], (5, 2): [2], (5, 3): [2], (5, 4): [2], (5, 5): [2], (5, 6): [2], (5, 7): [2], (6, 1): [3], (6, 2): [3], (6, 3): [3], (6, 4): [3], (6, 5): [3], (6, 6): [3], (6, 7): [3], (7, 1): [4], (7, 2): [4], (7, 3): [4], (7, 4): [4], (7, 5): [4], (7, 6): [4], (7, 7): [4], (8, 1): [5], (8, 2): [5], (8, 3): [5], (8, 4): [5], (8, 5): [5], (8, 6): [5], (8, 7): [5], (9, 1): [6], (9, 2): [6], (9, 3): [6], (9, 4): [6], (9, 5): [6], (9, 6): [6], (9, 7): [6], (10, 1): [7], (10, 2): [7], (10, 3): [7], (10, 4): [7], (10, 5): [7], (10, 6): [7], (10, 7): [7], (11, 1): [8], (11, 2): [8], (11, 3): [8], (11, 4): [8], (11, 5): [8],

In [5]:
def find_tradeoff_table(solver, obj1, obj2, obj1_bounds=obj1_bounds, obj2_bounds=obj2_bounds):
    
    ################################################################################
    #                               OBJ1                                           #
    ################################################################################
    # Solve
    print(f"Solving with {solver.SolverVersion()}")
    solver.Minimize(obj1)
    status = solver.Solve()

    # Print solution.

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f"Total cost = {obj1.solution_value()}")
        obj1_best = obj1.solution_value()
        obj2_worst = obj2.solution_value()
        print(f"Total penalty = {obj2.solution_value()}\n")
        
    else:
        print("No solution found.")

    
    ################################################################################
    #                               OBJ2                                           #
    ################################################################################
    # Set bounds
    ct_obj1.SetBounds(obj1_bounds[0], obj1_bounds[1])
    ct_obj2.SetBounds(obj2_bounds[1], obj2_bounds[0])
    
    # Solve
    print(f"Solving with {solver.SolverVersion()}")
    solver.Minimize(obj2)
    status = solver.Solve()

    # Print solution.

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f"Total cost = {obj1.solution_value()}")
        obj1_worst = obj1.solution_value()
        obj2_best = obj2.solution_value()
        print(f"Total penalty = {obj2.solution_value()}\n")
        
    else:
        print("No solution found.")
    
    print("obj1_worst", obj1_worst)
    print("obj1_best", obj1_best)
    print("obj2_worst", obj2_worst)
    print("obj2_best", obj2_best)

    return (obj1_best, obj1_worst), (obj2_best, obj2_worst)

obj1_bounds, obj2_bounds = find_tradeoff_table(solver, obj1, obj2)
find_tradeoff_table(solver, obj1, obj2)

In [6]:
def epsilon_constraints(solver, 
                        obj1=obj1, obj2=obj2, 
                        ct_obj1=ct_obj1, ct_obj2=ct_obj2, 
                        obj1_bounds=obj1_bounds, obj2_bounds=obj2_bounds, 
                        ProCost=ProCost, DistCost=DistCost,
                        num_eps=10):
    
    # Sets a time limit of 1 hour.
    solver.SetTimeLimit(10*60*1000)

    # set a minimum gap limit for the integer solution during branch and cut
    gap = 0.05
    solverParams = pywraplp.MPSolverParameters()
    solverParams.SetDoubleParam(solverParams.RELATIVE_MIP_GAP, gap)

    obj1 = [10, 50]
    obj2 = [2, 8]
    
    step2 = (8 - 2) / 3 = 2

    # Set epsilon gaps
    gap_eps1 = (obj1_bounds[1] - obj1_bounds[0]) / num_eps 
    gap_eps2 = (obj2_bounds[1] - obj2_bounds[0]) / num_eps 

    
    for eps in range(0, num_eps):

        print("epsilon:", eps+1)

        # Set epsilons bounds
        ct_obj1.SetBounds(obj1_bounds[1] - gap_eps1*(eps+1), obj1_bounds[1]- gap_eps1*eps)
        ct_obj1.set_is_lazy(laziness=True)

        ct_obj2.SetBounds(obj2_bounds[0]+ gap_eps2*eps, obj2_bounds[0] + gap_eps2*(eps+1))
        ct_obj2.set_is_lazy(laziness=True)

        # Solve
        solver.Minimize(obj1)
        status = solver.Solve()

        # Print solution.
        if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
            print(f"Total cost = {obj1.solution_value()}")
            print(f"Processing cost = {ProCost.solution_value()}")
            print(f"Distribution cost = {obj1.solution_value()-ProCost.solution_value()}")
            print(f"Total penalty = {obj2.solution_value()}\n")
            
        else:
            print("No solution found.\n")
    
    
epsilon_constraints(solver, num_eps=10)

epsilon: 1
Total cost = 280531.69489375
Processing cost = 233510.0
Distribution cost = 47021.69489375001
Total penalty = 27.300000000000004

epsilon: 2
Total cost = 277221.49969375
Processing cost = 233510.0
Distribution cost = 43711.49969375
Total penalty = 27.838633333333338

epsilon: 3
Total cost = 273911.30449375
Processing cost = 233510.0
Distribution cost = 40401.30449374998
Total penalty = 28.37726666666667

epsilon: 4
Total cost = 270601.10929375
Processing cost = 233510.0
Distribution cost = 37091.10929375002
Total penalty = 28.915900000000008

epsilon: 5
Total cost = 267290.91409375
Processing cost = 233510.0
Distribution cost = 33780.914093750005
Total penalty = 29.45453333333334

epsilon: 6
Total cost = 263980.71889375
Processing cost = 233510.0
Distribution cost = 30470.71889374999
Total penalty = 29.993166666666674

epsilon: 7
Total cost = 260670.52369375003
Processing cost = 233510.0
Distribution cost = 27160.52369375003
Total penalty = 30.531800000000004

epsilon: 8
Tot

In [7]:
def augumencon2(solver, obj1=obj1, obj2=obj2, 
                ct_obj1=ct_obj1, ct_obj2=ct_obj2, 
                obj1_bounds=obj1_bounds, obj2_bounds=obj2_bounds, 
                ProCost=ProCost, DistCost=DistCost,
                num_eps=20):

    # Sets a time limit of 1 hour.
    solver.SetTimeLimit(10*60*1000)

    # set a minimum gap limit for the integer solution during branch and cut
    gap = 0.05
    solverParams = pywraplp.MPSolverParameters()
    solverParams.SetDoubleParam(solverParams.RELATIVE_MIP_GAP, gap)

    # Set epsilon gaps
    gap_eps1 = (obj1_bounds[1] - obj1_bounds[0]) / num_eps  
    gap_eps2 = (obj2_bounds[1] - obj2_bounds[0]) / num_eps 

    # Define augumented paramters
    eps = 1e-5
    S2 = gap_eps2
    r2 = 11
    g2 = num_eps - 1
    i2 = 0
    

    while i2 < g2:
        
        print("epsilon:", i2+1)

        c1 = np.random.random(1)[0]/5*gap_eps1
        c2 = np.random.random(1)[0]/2*gap_eps2

        e1 = gap_eps1*i2 + c1
        e1_sub = gap_eps1*(i2+1) + c1

        e2 = gap_eps2*i2 + c2
        e2_sub = gap_eps2*(i2+1) + c2

        # Set epsilons bounds
        ct_obj1.SetBounds(obj1_bounds[1] - e1_sub, obj1_bounds[1]- e1)
        ct_obj1.set_is_lazy(laziness=True)

        ct_obj2.SetBounds(obj2_bounds[0] + e2, obj2_bounds[0] + e2_sub)
        ct_obj2.set_is_lazy(laziness=True)

        # Solve
        solver.Minimize(obj1)
        status = solver.Solve()

        # Print solution.
        if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
            print(f"Total cost = {obj1.solution_value()}")
            print(f"Processing cost = {ProCost.solution_value()}")
            print(f"Distribution cost = {obj1.solution_value()-ProCost.solution_value()}")
            print(f"Total penalty = {obj2.solution_value()}\n")
            b = int(S2 / gap_eps2)
            i2 += b
            
        else:
            print("No solution found.\n")
            # i2 = g2
            
augumencon2(solver, num_eps=10)


epsilon: 1
Total cost = 280323.0287885241
Processing cost = 233510.0
Distribution cost = 46813.0287885241
Total penalty = 27.38928288288007

epsilon: 2
Total cost = 276974.5088082133
Processing cost = 233510.0
Distribution cost = 43464.508808213286
Total penalty = 27.930472307695993

epsilon: 3
Total cost = 273901.351118017
Processing cost = 233510.0
Distribution cost = 40391.35111801699
Total penalty = 28.640618423366938

epsilon: 4
Total cost = 270319.0844522712
Processing cost = 233510.0
Distribution cost = 36809.08445227123
Total penalty = 29.021099348678447

epsilon: 5
Total cost = 266935.04409205436
Processing cost = 233510.0
Distribution cost = 33425.04409205436
Total penalty = 29.57991107341216

epsilon: 6
Total cost = 263492.4595581494
Processing cost = 233510.0
Distribution cost = 29982.459558149392
Total penalty = 30.181152150852213

epsilon: 7
Total cost = 260341.95739051144
Processing cost = 233510.0
Distribution cost = 26831.957390511438
Total penalty = 30.562141009150704